In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager #use to initialize driver in a better way without having chromedriver path mentioned.
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import os
import time
import ssl
from urllib.request import urlretrieve
import tkinter, tkinter.constants, tkinter.filedialog

In [ ]:
def whichDirectory():
    return tkinter.filedialog.askdirectory()

def inp(text):
    return input(text)

ssl._create_default_https_context = ssl._create_unverified_context

def scraper():
    try:
        chrome_options = Options()
        chrome_options.add_argument("--no-sandbox")
        driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options) 
        driver.maximize_window()
        for p in range(1, searchPage + 1):
            url = "https://www.shutterstock.com/search/" + searchTerm + "?image_type="
            driver.get(url)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight)") 
            time.sleep(5)                                                           
            data = driver.execute_script("return document.documentElement.outerHTML")
            print("Page " + str(p))
            BS = BeautifulSoup(data, "lxml")
            img_container = BS.find_all("img", {"class":"z_h_9d80b z_h_2f2f0"})
            for n in range(0, len(img_container)-1):
                img_src = img_container[n].get("src")
                name = img_src.rsplit("/", 1)[-1]
                try:
                    urlretrieve(img_src, os.path.join(scrape_directory, os.path.basename(img_src)))
                    print("Scraped " + name)
                except Exception as e:
                    print(e)
        driver.close()
    except Exception as e:
        print(e)
        
while True:
    while True:
        print("Please select a directory to save your scraped files.")
        scrape_directory = whichDirectory()
        if scrape_directory == None or scrape_directory == "":
            print("You must select a directory to save your scraped files.")
            continue
        break
    while True:
        start = inp("Would you like to begin scraping? ('y' for yes or 'n' for no): ")
        if start != "n" and start != "y":
            print("You must select 'y' if you would like to start scraping or 'n' to discontinue.")
            continue
        break
    if start == 'y':
        while True:
            image_type = inp("Select image type ('a' for all images or 'p' for photo): ")
            if image_type != "a" and image_type != "p":
                print("Please select 'a' for all images or 'p' for photo.")
                continue
            break
        if image_type == 'p':
            image_type = 'photo'
        else:
            image_type = 'all'
    if start == 'n':
        print("Scraping will not continue.")
        break
    while True:
        searchTerm = inp("What image would you like to scrape?")
        break
    while True:
        searchPage = int(input("How many pages would you like to scrape? "))
        if searchPage < 1:
            print("One page is the minimum. Please input the number of pages you would like to scrape.")
            continue
        break
    if start == "y":
        scraper()
    print("Successfully scraped")
    restartScrape = inp("Continue scraping? ('y' for yes or 'n' for no) ")
    if restartScrape == "n":
        print("Scraping ended.")
        break